In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet timm pytorch_lightning==1.7.7 torchmetrics==0.11.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.1/708.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 27.8 MB/s eta 0:00:00


In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pytorch_lightning as L
import zipfile, shutil

from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torchvision.transforms import v2 as  transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import RandomAffine, RandomHorizontalFlip, RandomVerticalFlip, ColorJitter
import timm

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, path_col,  mode='train'):
        self.df = df
        self.path_col = path_col
        self.mode = mode

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.mode == 'train':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'val':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            label = row['class']
            data = {
                'image':image,
                'label':label
            }
            return data
        elif self.mode == 'inference':
            row = self.df.iloc[idx]
            image = read_image(row[self.path_col])/256.
            data = {
                'image':image,
            }
            return data

    def train_transform(self, image):
        pass

In [ ]:
class CustomCollateFn:
    def __init__(self, transform, mode):
        self.mode = mode
        self.transform = transform

    def __call__(self, batch):
        if self.mode=='train':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            label = torch.LongTensor([data['label'] for data in batch])
            return {
                'pixel_values':pixel_values,
                'label':label,
            }
        elif self.mode=='val':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            label = torch.LongTensor([data['label'] for data in batch])
            return {
                'pixel_values':pixel_values,
                'label':label,
            }
        elif self.mode=='inference':
            pixel_values = torch.stack([self.transform(data['image']) for data in batch])
            return {
                'pixel_values':pixel_values,
            }

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model):
        super(CustomModel, self).__init__()
        self.model = model
        self.clf = nn.Sequential(
            nn.SiLU(),
            nn.LazyLinear(25),
        )

    def forward(self, x, label=None):
        x = self.model(x)
        logits = self.clf(x)
        loss = None
        if label is not None:
            loss = nn.CrossEntropyLoss()(logits, label)
        probs = nn.LogSoftmax(dim=-1)(logits)
        return probs, loss

class LitCustomModel(L.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = CustomModel(model)
        self.validation_step_output = []

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=1e-5)
        return opt

    def training_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        label = batch['label']
        probs, loss = self.model(x, label)
        self.log(f"train_loss", loss, on_step=True, on_epoch=False)
        return loss

    def validation_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        label = batch['label']
        probs, loss = self.model(x, label)
        self.validation_step_output.append([probs, label, loss])  # Add loss to validation_step_output
        return loss  # Return loss value

    def predict_step(self, batch, batch_idx=None):
        x = batch['pixel_values']
        probs, _ = self.model(x)
        return probs

    def validation_epoch_end(self, step_output):
        pred = torch.cat([x for x, _, _ in self.validation_step_output]).cpu().detach().numpy().argmax(1)
        label = torch.cat([label for _, label, _ in self.validation_step_output]).cpu().detach().numpy()
        score = f1_score(label, pred, average='macro')

        # Calculate validation loss
        val_loss = torch.stack([loss for _, _, loss in self.validation_step_output]).mean()

        self.log("val_score", score)
        self.log("val_loss", val_loss)  # Log validation loss
        self.validation_step_output.clear()
        return score


In [ ]:
SEED = 7
N_SPLIT = 5
BATCH_SIZE = 32

In [ ]:
L.seed_everything(SEED)

INFO:pytorch_lightning.utilities.seed:Global seed set to 7


7

In [ ]:
def dataset_extract(file_name, file_path) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if not os.path.exists(file_path) :
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=file_path)

In [ ]:
dataset_extract('/content/drive/MyDrive/open.zip', './')

Extracting:   0%|          | 0/38460 [00:00<?, ?files/s]

In [ ]:
train_df = pd.read_csv('./train.csv')
train_df.head()

,img_path,upscale_img_path,label
0,./train/TRAIN_00000.jpg,./upscale_train/TRAIN_00000.png,Ruddy Shelduck
1,./train/TRAIN_00001.jpg,./upscale_train/TRAIN_00001.png,Gray Wagtail
2,./train/TRAIN_00002.jpg,./upscale_train/TRAIN_00002.png,Indian Peacock
3,./train/TRAIN_00003.jpg,./upscale_train/TRAIN_00003.png,Common Kingfisher
4,./train/TRAIN_00004.jpg,./upscale_train/TRAIN_00004.png,Common Kingfisher


In [ ]:
train_df = pd.read_csv('./train.csv')
le = LabelEncoder()
train_df['class'] = le.fit_transform(train_df['label'])

In [ ]:
if not len(train_df) == len(os.listdir('./train')):
    raise ValueError()

In [ ]:
skf = StratifiedKFold(n_splits=N_SPLIT, random_state=SEED, shuffle=True)

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    RandomAffine(degrees=30, translate=(0.2, 0.2), scale=(0.8, 1.2)),
    RandomHorizontalFlip(p=0.5),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

val_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

train_collate_fn = CustomCollateFn(train_transform, 'train')
val_collate_fn = CustomCollateFn(val_transform, 'val')

- 아래 학습하는 코드는 3명이 학습한 모델을 앙상블 했기 때문에, for _ in range(3):을 써서 3번 학습했습니다.
- 따라서, 한번만 모델을 돌리고 싶은 경우 첫번째 for문을 빼주시면 됩니다!

In [ ]:
for _ in range(3):
    for fold_idx, (train_index, val_index) in enumerate(skf.split(train_df, train_df['class'])):
        train_fold_df = train_df.loc[train_index,:]
        val_fold_df = train_df.loc[val_index,:]

        train_dataset = CustomDataset(train_fold_df, 'img_path', mode='train')
        val_dataset = CustomDataset(val_fold_df, 'img_path', mode='val')

        train_dataloader = DataLoader(train_dataset, collate_fn=train_collate_fn, batch_size=BATCH_SIZE)
        val_dataloader = DataLoader(val_dataset, collate_fn=val_collate_fn, batch_size=BATCH_SIZE*2)

        model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
        lit_model = LitCustomModel(model)

        checkpoint_callback = ModelCheckpoint(
            monitor='val_score',
            mode='max',
            dirpath='./checkpoints/',
            filename=f'eva-large-196-resize-fold_idx={fold_idx}'+'-{epoch:02d}-{train_loss:.4f}-{val_loss:.4f}-{val_score:.4f}',
            save_top_k=1,
            save_weights_only=True,
            verbose=True
        )

        earlystopping_callback = EarlyStopping(monitor="val_score", mode="max", patience=6)
        trainer = L.Trainer(max_epochs=100, accelerator='auto', precision=32, callbacks=[checkpoint_callback, earlystopping_callback], val_check_interval=0.5)
        trainer.fit(lit_model, train_dataloader, val_dataloader)

        model.cpu()
        lit_model.cpu()
        del model, lit_model, checkpoint_callback, earlystopping_callback, trainer
        gc.collect()
        torch.cuda.empty_cache()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/model_summary/mo

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 7, global step 3168: 'val_score' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 3366: 'val_score' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 8, global step 3564: 'val_score' was not in top 1


In [ ]:
test_df = pd.read_csv('./test.csv')

In [ ]:
if not len(test_df) == len(os.listdir('./test')):
    raise ValueError()

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(size=(196,196), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

test_collate_fn = CustomCollateFn(test_transform, 'inference')
test_dataset = CustomDataset(test_df, 'img_path', mode='inference')
test_dataloader = DataLoader(test_dataset, collate_fn=test_collate_fn, batch_size=BATCH_SIZE*2)

In [ ]:
# 소프트보팅을 위한 예측
fold_probs = []
for checkpoint_path in glob('/content/checkpoints/*.ckpt'):
    model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
    lit_model = LitCustomModel.load_from_checkpoint(checkpoint_path, model=model)
    trainer = L.Trainer( accelerator='auto', precision=32)
    preds = trainer.predict(lit_model, test_dataloader)
    probs = F.softmax(torch.cat(preds, dim=0), dim=1).detach().cpu().numpy()  # 각 클래스에 대한 확률값으로 변환
    fold_probs.append(probs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_co

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [ ]:
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# 소프트보팅
# 각 모델에서 예측한 확률을 평균화하기 위해 모든 예측 확률을 쌓습니다.
pred_probs = np.stack(fold_probs, axis=0)

# 각 클래스에 대한 평균 예측 확률을 계산합니다.
mean_probs = np.mean(pred_probs, axis=0)

# 각 샘플에 대해 가장 높은 평균 확률을 가진 클래스를 선택합니다.
pred_ensemble_soft = np.argmax(mean_probs, axis=1)

submission['label'] = le.inverse_transform(pred_ensemble_soft)

In [ ]:
submission.to_csv('/content/drive/MyDrive/AIVLE/dacon/eva_large_196_silu/eva_large_196_silu_softvoting_submit.csv',index=False)

# 제출한 모델 재현
- 아래 코드는 저희가 최종 제출한 모델의 가중치를 다운로드, 압축 해제해서 예측한 코드입니다.
- 최종 Private 점수 재현은 아래 코드로 해주시면 감사하겠습니다.

## 체크포인트 압축파일 다운로드 및 해제
## 압축파일 다운로드

In [ ]:
import gdown
import glob

In [ ]:
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

In [ ]:
checkpoint_file_list = ['1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE', '10abqWneGvnpyc9ZWhkw8a2svcdhTKWwI', '1Ho4KpxpHLGKN3yRDYih6wcEPLuRZWmu8']
for i, checkpoint in zip(range(1,len(checkpoint_file_list)+1), checkpoint_file_list):
    download_file(checkpoint, f'checkpoint_{i}.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE
From (redirected): https://drive.google.com/uc?id=1XxX5e2shpxIQt6tnrXaBxdP4ue4HfcfE&confirm=t&uuid=b36f6929-43a3-4c59-b9f1-dd7091133283
To: /content/checkpoint_1.zip
100%|██████████| 5.66G/5.66G [01:02<00:00, 91.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=10abqWneGvnpyc9ZWhkw8a2svcdhTKWwI
From (redirected): https://drive.google.com/uc?id=10abqWneGvnpyc9ZWhkw8a2svcdhTKWwI&confirm=t&uuid=c43546ef-b651-4388-aa23-2e99c2a1887f
To: /content/checkpoint_2.zip
100%|██████████| 5.66G/5.66G [00:48<00:00, 116MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1Ho4KpxpHLGKN3yRDYih6wcEPLuRZWmu8
From (redirected): https://drive.google.com/uc?id=1Ho4KpxpHLGKN3yRDYih6wcEPLuRZWmu8&confirm=t&uuid=648ecda3-26cc-4182-aeac-fa4c5f0b624a
To: /content/checkpoint_3.zip
100%|██████████| 5.66G/5.66G [00:37<00:00, 151MB/s] 


## 압축파일 해제

In [ ]:
import subprocess

def extract_archive(file_name, output_path):
    os.makedirs(output_path, exist_ok=True)
    subprocess.run(['unzip', '-o', file_name, '-d', output_path])

for zip_path in glob.glob('/content/*.zip'):
    print(f'Extracting: {zip_path}')
    extract_archive(zip_path, './')

Extracting: /content/checkpoint_2.zip
Extracting: /content/checkpoint_1.zip
Extracting: /content/checkpoint_3.zip


## 소프트보팅

In [ ]:
# 소프트보팅을 위한 예측
fold_probs = []
for checkpoint_path in glob.glob('/content/*ckpt'):
    model = timm.create_model('eva_large_patch14_196.in22k_ft_in22k_in1k', pretrained=True)
    lit_model = LitCustomModel.load_from_checkpoint(checkpoint_path, model=model)
    trainer = L.Trainer( accelerator='auto', precision=32)
    preds = trainer.predict(lit_model, test_dataloader)
    probs = F.softmax(torch.cat(preds, dim=0), dim=1).detach().cpu().numpy()  # 각 클래스에 대한 확률값으로 변환
    fold_probs.append(probs)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_co

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

## 제출파일 불러오기

In [ ]:
submission = pd.read_csv('./sample_submission.csv')

In [ ]:
# 소프트보팅
# 각 모델에서 예측한 확률을 평균화하기 위해 모든 예측 확률을 쌓습니다.
pred_probs = np.stack(fold_probs, axis=0)

# 각 클래스에 대한 평균 예측 확률을 계산합니다.
mean_probs = np.mean(pred_probs, axis=0)

# 각 샘플에 대해 가장 높은 평균 확률을 가진 클래스를 선택합니다.
pred_ensemble_soft = np.argmax(mean_probs, axis=1)

submission['label'] = le.inverse_transform(pred_ensemble_soft)

In [ ]:
submission.to_csv('/content/drive/MyDrive/AIVLE/dacon/eva_large_196_silu/eva_large_196_silu_softvoting_submit_final.csv',index=False)